In [1]:
%load_ext tensorboard
import datetime

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers

import tensorflow_datasets as tfds

from tensorflow.data import AUTOTUNE

The tensorboard module is not an IPython extension.


C:\Users\andre\anaconda3\envs\Py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\andre\anaconda3\envs\Py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\andre\anaconda3\envs\Py3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\andre\anaconda3\envs\Py3.6\lib\site-packages\tensorflow\python\f

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3000)])
  except RuntimeError as e:
    print(e)

Found GPU at: /device:GPU:0


In [3]:
import tensorflow as tf

import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.metrics as metrics
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras import regularizers
import tensorflow_addons as tfa
from typing import Callable


# default input shape
INPUT_SHAPE = (512, 1024, 3)


def ge_layer(x_in, c, e=6, stride=1):
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x_in)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    if stride == 2:
        x = layers.DepthwiseConv2D(depth_multiplier=e, kernel_size=(3,3), strides=2, padding='same')(x)
        x = layers.BatchNormalization()(x)
        
        y = layers.DepthwiseConv2D(depth_multiplier=e, kernel_size=(3,3), strides=2, padding='same')(x_in)
        y = layers.BatchNormalization()(y)
        y = layers.Conv2D(filters=c, kernel_size=(1,1),kernel_regularizer=regularizers.l2(5e-4), padding='same')(y)
        y = layers.BatchNormalization()(y)
    else:
        y = x_in
        
    x = layers.DepthwiseConv2D(depth_multiplier=e, kernel_size=(3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=c, kernel_size=(1,1),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Add()([x, y])
    x = layers.Activation('relu')(x)
    return x


def stem(x_in, c):
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), strides=2, padding='same')(x_in)
    x = layers.BatchNormalization()(x)
    x_split = layers.Activation('relu')(x)
    
    #x = layers.MaxPool2D(pool_size=(1, 1), padding='same')(x_split)
    x = layers.Conv2D(filters=c // 2, kernel_size=(1,1),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x_split)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    y = layers.MaxPooling2D()(x_split)
    
    x = layers.Concatenate()([x, y])
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    return x


def detail_conv2d(x_in, c, stride=1):
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), strides=stride, padding='same')(x_in)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    return x


def context_embedding(x_in, c):
    x = layers.GlobalAveragePooling2D()(x_in)
    x = layers.BatchNormalization()(x)
    
    x = layers.Reshape((1,1,c))(x)
    
    x = layers.Conv2D(filters=c, kernel_size=(1,1),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    # broadcasting no needed
    print(x_in)
    print(x)
    x = layers.Add()([x, x_in])
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x)
    return x


def bilateral_guided_aggregation(detail, semantic, c):
    # detail branch
    detail_a = layers.DepthwiseConv2D(kernel_size=(3,3), padding='same')(detail)
    detail_a = layers.BatchNormalization()(detail_a)
    
    detail_a = layers.Conv2D(filters=c, kernel_size=(1,1),kernel_regularizer=regularizers.l2(5e-4), padding='same')(detail_a)
    
    #detail_a = layers.MaxPool2D(pool_size=(3,3), strides=2, padding='same')(detail_a)
    
    detail_b = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), strides=2, padding='same')(detail)
    detail_b = layers.BatchNormalization()(detail_b)
    
    detail_b = layers.AveragePooling2D((3,3), strides=2, padding='same')(detail_b)
    
    # semantic branch
    semantic_a = layers.DepthwiseConv2D(kernel_size=(3,3), padding='same')(semantic)
    semantic_a = layers.BatchNormalization()(semantic_a)
    
    #semantic_a = layers.MaxPool2D(pool_size=(1, 1), padding='same')(semantic_a)
    semantic_a = layers.Conv2D(filters=c, kernel_size=(1,1),kernel_regularizer=regularizers.l2(5e-4), padding='same')(semantic_a)
    semantic_a = layers.Activation('sigmoid')(semantic_a)
    
    semantic_b = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(semantic)
    semantic_b = layers.BatchNormalization()(semantic_b)
    
    #semantic_b = layers.MaxPool2D(pool_size=(3,3), strides=2, padding='same')(semantic_b)
    
    semantic_b = layers.UpSampling2D((4,4), interpolation='bilinear')(semantic_b)
    semantic_b = layers.Activation('sigmoid')(semantic_b)
    
    # combining
    detail = layers.Multiply()([detail_a, semantic_b])
    semantic = layers.Multiply()([semantic_a, detail_b])
    
    # this layer is not mentioned in the paper !?
    #semantic = layers.UpSampling2D((4,4))(semantic)
    semantic = layers.UpSampling2D((4,4), interpolation='bilinear')(semantic)
    
    x = layers.Add()([detail, semantic])
    x = layers.Conv2D(filters=c, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    return x


def seg_head(x_in, c_t, s, n):
    x = layers.Conv2D(filters=c_t, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x_in)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=n, kernel_size=(3,3),kernel_regularizer=regularizers.l2(5e-4), padding='same')(x)
    x = layers.UpSampling2D((s,s), interpolation='bilinear')(x)
    
    return x


class ArgmaxMeanIOU(metrics.MeanIoU):
    def update_state(self, y_true, y_pred, sample_weight=None):
        return super().update_state(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1), sample_weight)


def bisenetv2(num_classes=2, out_scale=8, input_shape=INPUT_SHAPE, l=4, seghead_expand_ratio=2):
    x_in = layers.Input(input_shape)

    # semantic branch
    # S1 + S2
    x = stem(x_in, 16 // 1)
    
    # S3
    x = ge_layer(x, 32 // 1, stride=2)
    x = ge_layer(x, 32 // 1, stride=1)

    # S4
    #x = ge_layer(x, 64, stride=2)
    #x = ge_layer(x, 64, stride=1)

    # S5
    #x = ge_layer(x, 128, stride=2)

    #x = ge_layer(x, 128, stride=1)
    #x = ge_layer(x, 128, stride=1)
    #x = ge_layer(x, 128, stride=1)

    #x = context_embedding(x, 128)
    x = context_embedding(x, 32)

    # detail branch
    # S1
    y = detail_conv2d(x_in, 64, stride=2)
    y = detail_conv2d(y, 64, stride=1)

    # S2
    #y = detail_conv2d(y, 64, stride=2)
    #y = detail_conv2d(y, 64, stride=1)
    #y = detail_conv2d(y, 64, stride=1)

    # S3
    #y = detail_conv2d(y, 128, stride=2)
    #y = detail_conv2d(y, 128, stride=1)
    #y = detail_conv2d(y, 128, stride=1)

    #x = bilateral_guided_aggregation(y, x, 128)
    x = bilateral_guided_aggregation(y, x, 64)

    x = seg_head(x, num_classes * seghead_expand_ratio, out_scale, num_classes)
    
    model = models.Model(inputs=[x_in], outputs=[x])
    
    # set weight initializers
    for layer in model.layers:
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel_initializer = tf.keras.initializers.HeNormal()
        if hasattr(layer, 'depthwise_initializer'):
            layer.depthwise_initializer = tf.keras.initializers.HeNormal()

    return model


def bisenetv2_compiled(num_classes, warmup_steps, decay_steps=5e4, momentum=0.9, weight_decay=0.0005, warmup=True, moving_avg=True, opt_type=1, lr_type=0, **kwargs):
    model = bisenetv2(num_classes, **kwargs)

    poly = optimizers.schedules.PolynomialDecay(
        initial_learning_rate=5e-2,
        decay_steps=decay_steps,
        power=0.9
    )
    
    cosine = optimizers.schedules.CosineDecay(
        initial_learning_rate = 5e-2, decay_steps=decay_steps, alpha=0.0)
    
    poly_cycle = optimizers.schedules.PolynomialDecay(
        initial_learning_rate=5e-2,
        decay_steps=decay_steps,
        power=0.9,
        cycle = True)
    
    lr = [poly, cosine, poly_cycle]
    schedule = lr[lr_type]
    
    if warmup:
        lr_schedule = WarmUp(
        initial_learning_rate=5e-2,
        decay_schedule_fn=schedule,
        warmup_steps=warmup_steps)
    else:
        lr_schedule = schedule
    
    sgdw = tfa.optimizers.SGDW(
            weight_decay=weight_decay,
            learning_rate=lr_schedule,
            momentum=momentum
        )
    sgd = optimizers.SGD(
            learning_rate=lr_schedule,
            momentum=momentum,
        )
    
    adamw = tfa.optimizers.AdamW(learning_rate=lr_schedule , weight_decay=weight_decay)
    
    optimizer = [sgdw, sgd, adamw]
    
    if moving_avg:
        opt = tfa.optimizers.MovingAverage(optimizer[opt_type], start_step=warmup_steps, dynamic_decay=True)
    opt = optimizer[opt_type]
    cce = losses.CategoricalCrossentropy(from_logits=True)
    model.compile(opt, loss=cce,
                  metrics=['accuracy', ArgmaxMeanIOU(num_classes)]) 
    
    return model

class WarmUp(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(
    self,
    initial_learning_rate: float,
    decay_schedule_fn: Callable,
    warmup_steps: int,
    power: float = 1.0,
    name: str = None,
    ):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.warmup_steps = warmup_steps
        self.power = power
        self.decay_schedule_fn = decay_schedule_fn
        self.name = name
    
    def __call__(self, step):
        with tf.name_scope(self.name or "WarmUp") as name:
            # Implements polynomial warmup. i.e., if global_step < warmup_steps, the
            # learning rate will be `global_step/num_warmup_steps * init_lr`.
            global_step_float = tf.cast(step, tf.float32)
            warmup_steps_float = tf.cast(self.warmup_steps, tf.float32)
            warmup_percent_done = global_step_float / warmup_steps_float
            warmup_learning_rate = (self.initial_learning_rate) * tf.math.pow(warmup_percent_done, self.power)
            return tf.cond(
                global_step_float < warmup_steps_float,
                lambda: warmup_learning_rate,
                lambda: self.decay_schedule_fn(step - self.warmup_steps),
                name=name,
            )

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_schedule_fn": self.decay_schedule_fn,
            "warmup_steps": self.warmup_steps,
            "power": self.power,
            "name": self.name,
        }


In [4]:
import json
import tensorflow as tf
import numpy as np
import cv2


seed = 123

Crop_img = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomCrop(128, 256, seed=seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])
Crop_seg= tf.keras.Sequential([
  layers.experimental.preprocessing.RandomCrop(128, 256, seed=seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])

seed = 456

Zoom_img = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomZoom((-0.75,0.25), fill_mode='constant', seed = seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])
Zoom_seg = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomZoom((-0.75,0.25), fill_mode='constant', seed = seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])

seed= 789

Flip_img = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])
Flip_seg = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])

def class_map_road(seg):
    # map class 0=anything, 1=road
    return tf.where(seg == 7, [0, 1.0], [1.0, 0])


def cityscapes_prep(output_shape, input_shape=INPUT_SHAPE, class_map_func=None, float_range=True, standardize=True):
    def prep_map(sample):
        img = sample['image_left']
        seg = sample['segmentation_label']

        if float_range:
            img /= 255
        if standardize:
            img = tf.image.per_image_standardization(img)

        img = tf.image.resize(img, input_shape[0:2], method='bilinear')
        seg = tf.image.resize(seg, output_shape[0:2], method='nearest')
        
        img = img[None, ...]
        seg = seg[None, ...]
        img = Flip_img(img)
        seg = Flip_seg(seg)
        img = tf.squeeze(img)
        seg = tf.squeeze(seg,0)
        
        if callable(class_map_func):
            seg = class_map_func(seg)
        else:
            seg = tf.one_hot(tf.cast(seg, tf.int32), output_shape[-1], axis=2)
            seg = tf.cast(seg, tf.float32)
            seg = tf.squeeze(seg)
            #seg = tf.keras.utils.to_categorical(seg, num_classes=output_shape[-1])

        return img, seg

    return prep_map

def Zoom_prep(output_shape, input_shape=INPUT_SHAPE, class_map_func=None, float_range=True, standardize=True):
    def prep_map(sample):
        img = sample['image_left']
        seg = sample['segmentation_label']

        if float_range:
            img /= 255
        if standardize:
            img = tf.image.per_image_standardization(img)

        img = tf.image.resize(img, input_shape[0:2], method='bilinear')
        seg = tf.image.resize(seg, output_shape[0:2], method='nearest')
        
        img = img[None, ...]
        seg = seg[None, ...]
        img = Zoom_img(img)
        seg = Zoom_seg(seg)
        img = tf.squeeze(img)
        seg = tf.squeeze(seg,0)
        
        if callable(class_map_func):
            seg = class_map_func(seg)
        else:
            seg = tf.one_hot(tf.cast(seg, tf.int32), output_shape[-1], axis=2)
            seg = tf.cast(seg, tf.float32)
            seg = tf.squeeze(seg)
            #seg = tf.keras.utils.to_categorical(seg, num_classes=output_shape[-1])
        print(seg)
        return img, seg

    return prep_map

def Crop_prep(output_shape, input_shape=INPUT_SHAPE, class_map_func=None, float_range=True, standardize=True, Training = True):
    def prep_map(sample):
        img = sample['image_left']
        seg = sample['segmentation_label']

        if float_range:
            img /= 255
        if standardize:
            img = tf.image.per_image_standardization(img)

        img = tf.image.resize(img, (200,400), method='bilinear')
        seg = tf.image.resize(seg, (200,400), method='nearest')
                    
        
        img = img[None, ...]
        seg = seg[None, ...]
        img = Crop_img(img)
        seg = Crop_seg(seg)
        img = tf.squeeze(img)
        seg = tf.squeeze(seg,0)
        
        if callable(class_map_func):
            seg = class_map_func(seg)
        else:
            seg = tf.one_hot(tf.cast(seg, tf.int32), output_shape[-1], axis=2)
            seg = tf.cast(seg, tf.float32)
            seg = tf.squeeze(seg)
            #seg = tf.keras.utils.to_categorical(seg, num_classes=output_shape[-1])
        
        return img, seg

    return prep_map

rng = tf.random.Generator.from_seed(123, alg='philox')

def augment(img, seg, seed):
        #seed = tf.random.uniform((2,))
        print(seg)
        img = tf.image.stateless_random_brightness(img, 0.2, seed=seed)
        img = tf.image.stateless_random_contrast(img, 0.8, 1.2, seed=seed)
        img = tf.clip_by_value(img, 0.0, 1.0) # clip values outside 0..1

        return img, seg

def randomize(img, seg):
        seed = rng.make_seeds(2)[0]
        img, seg = augment(img, seg, seed)
        return img, seg

In [5]:
import matplotlib.pyplot as plt


def segmentation_to_image(pred):
    img = tf.argmax(pred, axis=-1)
    img = img[..., tf.newaxis]
    return tf.keras.preprocessing.image.array_to_img(img)

        
def predict_tf(model):
    def predict_func(sample):
        print(sample[0])
        pred = model.predict(tf.expand_dims(sample[0], axis=0))
        return sample[0], pred[0]
    
    return predict_func


def display_dataset(ds, pred_func):
    for sample in ds:
        imgs = pred_func(sample)
        fig, axes = plt.subplots(1, len(imgs))
        
        for ax, img in zip(axes, imgs):
            if img.shape[-1] != 3:
                img = segmentation_to_image(img)
                
            ax.imshow(img)
            plt.show()

In [6]:
INPUT_SHAPE = (128, 256, 3)
OUTPUT_SHAPE = (128, 256, 2)
NUM_CLASSES = 2
SCALE = 2
m = bisenetv2_compiled(num_classes=NUM_CLASSES, out_scale=SCALE, input_shape=INPUT_SHAPE, warmup=True, opt_type=1, lr_type=1, warmup_steps=5*744)
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 128, 16)  448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 128, 16)  64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 128, 16)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [8]:
tf.random.set_seed(1234)

cityscapes = tfds.load('cityscapes/semantic_segmentation',data_dir="E:\EE\project\FPGA\cityscapes",download=False,shuffle_files=True)
train_ds = cityscapes['train'].map(cityscapes_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
valid_ds = cityscapes['validation'].map(cityscapes_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
test_ds = cityscapes['test'].map(cityscapes_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
crop_train = cityscapes['train'].map(Crop_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
zoom_train = cityscapes['train'].map(Zoom_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
tmp_ds = train_ds.concatenate(crop_train).concatenate(zoom_train)
train_rand = tmp_ds.map(randomize)
fit_ds = tmp_ds.concatenate(train_rand)

fit_ds = fit_ds.batch(16).prefetch(AUTOTUNE)
valid_ds = valid_ds.batch(4).prefetch(AUTOTUNE)
test_ds = test_ds.batch(4).prefetch(AUTOTUNE)
print(fit_ds)

Tensor("SelectV2:0", shape=(128, 256, 2), dtype=float32)
Tensor("args_1:0", shape=(128, 256, 2), dtype=float32)
<PrefetchDataset shapes: ((None, 128, 256, 3), (None, 128, 256, 2)), types: (tf.float32, tf.float32)>


In [ ]:
BATCH_SIZE = 16
  
USE_AUGMENTATION = True
rng = tf.random.Generator.from_seed(123, alg='philox')
    
if USE_AUGMENTATION:

    def augment(img, seg, seed):
        #seed = tf.random.uniform((2,))
        print(seg)
        img = tf.image.stateless_random_brightness(img, 0.2, seed=seed)
        img = tf.image.stateless_random_contrast(img, 0.8, 1.2, seed=seed)
        img = tf.image.stateless_random_saturation(img, 0.0, 1.5, seed=seed)
        img = tf.image.stateless_random_hue(img, 0.2, seed=seed)
        img = tf.clip_by_value(img, 0.0, 1.0) # clip values outside 0..1

        return img, seg

    def randomize(img, seg):
        seed = rng.make_seeds(2)[0]
        img, seg = augment(img, seg, seed)
        return img, seg
    
    train_ds_flipped_lr = train_ds.map(lambda img, seg: (tf.image.flip_left_right(img), tf.image.flip_left_right(seg)))
    valid_ds_flipped_lr = valid_ds.map(lambda img, seg: (tf.image.flip_left_right(img), tf.image.flip_left_right(seg)))
    
    
    
    train_ds_randomized = train_ds.map(randomize)
    valid_ds_randomized = valid_ds.map(randomize)
    
    train_ds_randomized_flipped = train_ds_flipped_lr.map(randomize)
    valid_ds_randomized_flipped = valid_ds_flipped_lr.map(randomize)
    
    train_ds = train_ds.concatenate(train_ds_flipped_lr).concatenate(train_ds_randomized).concatenate(train_ds_randomized_flipped)
    valid_ds = valid_ds.concatenate(valid_ds_flipped_lr).concatenate(valid_ds_randomized).concatenate(valid_ds_randomized_flipped)

    #train_ds = train_ds.concatenate(train_ds_flipped_lr).concatenate(train_ds_randomized).concatenate(crop_ds_flipped).concatenate(train_ds_randomized).concatenate(zoom_ds_flipped)
    #train_ds = train_ds.concatenate(aug_ds).concatenate(train_ds_randomized).concatenate(aug_ds_rand)
    
train_ds = train_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
valid_ds = valid_ds.batch(4).prefetch(AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

#display_dataset(valid_ds.unbatch().take(1), lambda s: (s[0], s[1]))

In [ ]:
EPOCHS = 100

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = m.fit(fit_ds,
                validation_data=valid_ds,
                epochs=EPOCHS,
                callbacks=[tensorboard_callback])

C:\Users\andre\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/100
1116/1116 [==============================] - 1599s 1s/step - loss: 0.6348 - accuracy: 0.8636 - argmax_mean_iou: 0.7473 - val_loss: 2.0063 - val_accuracy: 0.7613 - val_argmax_mean_iou: 0.5295
Epoch 2/100
1116/1116 [==============================] - 1649s 1s/step - loss: 0.4635 - accuracy: 0.9073 - argmax_mean_iou: 0.8203 - val_loss: 0.8653 - val_accuracy: 0.8316 - val_argmax_mean_iou: 0.6801
Epoch 3/100
1116/1116 [==============================] - 1667s 1s/step - loss: 0.3326 - accuracy: 0.9192 - argmax_mean_iou: 0.8412 - val_loss: 0.5165 - val_accuracy: 0.8662 - val_argmax_mean_iou: 0.7417
Epoch 4/100
1116/1116 [==============================] - 1697s 2s/step - loss: 0.2500 - accuracy: 0.9260 - argmax_mean_iou: 0.8533 - val_loss: 0.4590 - val_accuracy: 0.8526 - val_argmax_mean_iou: 0.7132
Epoch 5/100
1116/1116 [==============================] - 1676s 2s/step - loss: 0.2087 - accuracy: 0.9322 - argmax_mean_iou: 0.8648 - val_loss: 0.5050 - val_accuracy: 0.7653 - val_argmax_me

In [ ]:
m.save('E:/EE/project/FPGA/benchmark_model/model_benchmark_1.tf')
m.save('E:/EE/project/FPGA/benchmark_model/model_benchmark_1.h5',save_format='h5')
model_copy = tf.keras.models.clone_model(m)
model_copy.set_weights(m.get_weights())

In [ ]:
import cv2
import time
def img_pred(src,model,image_size):
    src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
    #cv2_imshow(src)
    image = cv2.resize(src,image_size,interpolation=cv2.INTER_CUBIC)
    #cv2_imshow(src)
    image = image/255
    image =  tf.image.per_image_standardization(image)
    data = tf.convert_to_tensor(image, dtype=tf.float32)
    data = tf.expand_dims(data, axis=0)
    s = time.time()
    pred = model.predict(data)
    e = time.time()
    fps = 1/(e-s)
    seg = tf.argmax(pred[0], axis=-1)
    seg = seg[..., tf.newaxis]
    seg = tf.keras.preprocessing.image.array_to_img(seg)
    #plt.imshow(seg)
    seg = cv2.cvtColor(np.array(seg), cv2.COLOR_BGR2RGB)
    seg = cv2.resize(seg,(src.shape[1],src.shape[0]),interpolation=cv2.INTER_CUBIC)
    result = cv2.addWeighted(np.array(src), 0.6, seg, 0.5, 0, dtype = cv2.CV_8U)
    return src, seg, result, fps

In [ ]:
import statistics as stat
from tqdm import notebook

import cv2
import time
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

def gen_seg_vid(cap,out,model,subtitle):
    length = 1000
    progress = notebook.tqdm(total = length)
    AVG_FPS = 0
  #while(cap.isOpened()):
    for _ in range(length):
        ret, frame = cap.read()
        cnt = 0
        if ret == True:
            cnt += 1
            total_fps = []
            m = model
            shape = (256,128)
            result = img_pred(frame,m,shape)
            img_seg = result[2]
            fps = result[3]
            total_fps.append(fps)
            if cnt == 10:
                AVG_FPS = stat.mean(total_fps)
                cnt = 0 
            x = 'AVG_FPS:'
            y = 'FPS: '
            img_seg = cv2.resize(img_seg,(512,256),interpolation=cv2.INTER_CUBIC)
            text = "{}{:.3f}".format(x,AVG_FPS)
            text1 = "{}{:.3f}".format(y,fps)
            font = cv2.FONT_HERSHEY_DUPLEX
            img_out = cv2.cvtColor(img_seg, cv2.COLOR_BGR2RGB)
            cv2.putText(img_out, text , (350,230), font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(img_out, text1 , (350,210), font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(img_out, subtitle , (140,20), font, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
            img_out = cv2.resize(img_out,(1800,1300),interpolation=cv2.INTER_CUBIC)
            
            out.write(img_out)
            progress.update(1)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return AVG_FPS

In [ ]:
model = tf.keras.models.load_model('E:/EE/project/FPGA/benchmark_model/model_benchmark1.tf',custom_objects={'ArgmaxMeanIOU': ArgmaxMeanIOU}, compile=False)
cap = cv2.VideoCapture('E:\EE\project\FPGA\Driving_data.mp4')
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('E:/EE/project/FPGA/benchmark_model/City_GPU_1.mp4',fourcc,20,(1800,1300))

subtitle = 'Benchmark mIoU(92%)'

gen_seg_vid(cap,out,model,subtitle)

In [ ]:
src = cv2.imread('E:/EE/project/FPGA/test_02.png')
start = time.time()
result = img_pred(src,model,(256,128))[2]
end = time.time()
fps = 1/(end-start)
print('fps =',fps)
result = cv2.resize(result,(1024,512),interpolation=cv2.INTER_CUBIC)
cv2.imshow("result",result)
cv2.waitKey (0)